In [0]:
%%writefile requirements.txt
torch
numpy
pandas
scikit-learn
razdel
ipymarkup

Writing requirements.txt


In [0]:
!pip install --upgrade -r requirements.txt

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 1)) (1.3.1)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (1.17.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 3)) (0.25.3)
     |████████████████████████████████| 7.0MB 49.7MB/s 
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


Новый день - новый датасет!

In [0]:
!wget http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip
!unzip Persons-1000.zip

--2019-12-17 08:04:14--  http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip
Resolving ai-center.botik.ru (ai-center.botik.ru)... 95.129.138.2
Connecting to ai-center.botik.ru (ai-center.botik.ru)|95.129.138.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3363777 (3.2M) [application/zip]
Saving to: ‘Persons-1000.zip’

Persons-1000.zip    100%[===================>]   3.21M  2.65MB/s    in 1.2s    

2019-12-17 08:04:16 (2.65 MB/s) - ‘Persons-1000.zip’ saved [3363777/3363777]

Archive:  Persons-1000.zip
   creating: Persons-1000/
   creating: Persons-1000/collection/
   creating: Persons-1000/collection/001/
  inflating: Persons-1000/collection/001/anno.markup.xml  
  inflating: Persons-1000/collection/001/text.txt  
   creating: Persons-1000/collection/002/
  inflating: Persons-1000/collection/002/anno.markup.xml  
  inflating: Persons-1000/collection/002/text.txt  
   creating: Persons-1000/collection/003/
 extracting: Persons-1000/collection/003/ann

In [0]:
!cat Persons-1000/collection/001/anno.markup.xml  

﻿<markup>
<entry>
<id>1</id>
<offset>308</offset>
<length>16</length>
<class>AAA_Estimate_Person</class>
<attribute>
<name>Canonical</name>
<value>ГРИГОРИЙ КАРАСИН</value>
</attribute>
</entry>
<entry>
<id>2</id>
<offset>387</offset>
<length>15</length>
<class>AAA_Estimate_Person</class>
<attribute>
<name>Canonical</name>
<value>ДЭНИЭЛ ФРИД</value>
</attribute>
</entry>
</markup>


# NER

## Датасет

Named Entity Recognition - распознавание именных сущностей. Выделяем в тексте спаны PER, LOC, ORG.

В случае с Persons-1000 только PER. 

In [0]:
import os
import xml.etree.ElementTree as ET
from ipymarkup import show_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN

directory = "Persons-1000/collection/"

def read_text_with_markup(directory):
    markup_file_name = os.path.join(directory, "anno.markup.xml")
    text_file_name = os.path.join(directory, "text.txt")
    with open(text_file_name, "r", encoding="windows-1251") as r:
        text = r.read()
    text = text.replace("\n", "\r\n")
    root = ET.parse(markup_file_name).getroot()
    spans = []
    for entry in root.findall("entry"):
        start_pos = int(entry.find("offset").text)
        end_pos = start_pos + int(entry.find("length").text)
        tag = entry.find("class").text
        spans.append((start_pos, end_pos, "PER"))
    return text, spans

data = []
for sample_name in os.listdir(directory):
    sample_path = os.path.join(directory, sample_name)
    data.append(read_text_with_markup(sample_path))

ipymarkup - модуль для вывода NER разметки в ipynb

In [0]:
show_box_markup(data[0][0], data[0][1], palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))

BoxMarkup('\r\nМарио Монти подаст в отставку с должности премьер-министра Италии\r\nФото: Reuters Фото: Reuters Фото: Reuters\r\nФото: Reuters\r\n\r\nПремьер-министр Италии Марио Монти подтвердил намерение подать прошение об отставке после того, как парламент одобрит закон о государственном бюджете на 2013 г. Этот и другие вопросы глава кабинета обсудил в ходе двухчасовой беседы с Президентом страны Джорджо Наполитано, передает Reuters.\r\n\r\nОтметим, что ранее входившая в парламентскую коалицию партия "Народ свободы" отказала Монти в поддержке. В свою очередь 76-летний Сильвио Берлускони, который является лидером указанной партии, за несколько часов до заявления Монти сообщил прессе, что сохраняет премьерские амбиции и готов вновь бороться за пост главы правительства республики в 2013 году.\r\n\r\nВ случае если бюджет будет принят "быстро", Монти готов немедленно сложить с себя полномочия. Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов. Несмотря на то, что такой вариант является маловероятным, итальянские политологи рассматривают его в качестве возможного исхода ситуации.\r\n\r\nПолитологи заключили, что теперь Берлускони обрушится с жесткой критикой на действующий кабинет министров, проводящий политику жесткой экономии.\r\n\r\n    Читайте также: Берлускони намерен в 2013 г. снова баллотироваться на пост премьер-министра Италии\r\n\r\nБерлускони ушел в отставку немногим более года назад, после чего в стране с согласия парламентского большинства было составлено правительство технократов, которому было поручено проводить непопулярную экономическую политику.\r\n\r\nКак сообщалось, 26 октября с.г. суд приговорил Берлускони к году тюрьмы с учетом закона об амнистии. Отмечается, что документ был принят в 2006 г., и в соответствии с ним срок тюремного заключения бывшего итальянского премьера уменьшен с четырех лет до одного года. Речь идет о скандальном деле Mediaset, в рамках которого Берлускони обвинялся в масштабных финансовых махинациях. Берлускони также отрицает обвинения в сексе с несовершеннолетней проституткой, известной под прозвищем "Руби".\r\n\r\n    Читайте также: Адвокаты Берлускони обжалуют приговор экс-премьеру\r\n\r\nОтметим также, действующий секретарь левоцентристской Демократической партии Италии Пьер Луиджи Берсани станет кандидатом на пост премьер-министра страны. Именно его кандидатуру поддержало большинство сторонников партии на праймериз 2 декабря. На выборах Берсани обошел мэра Флоренции Маттео Ренци, считающегося одним из наиболее заметных политиков "новой волны" в Италии. Демократическая партия, созданная на основе коалиции "Оливковое дерево" бывшего премьер-министра страны Романо Проди и включающая в себя партии "Левые демократы" и "Маргерита", занимает 217 мест в 630-местном парламенте Италии и является одной из главных оппозиционных сил страны.\r\n\r\n    По теме: В Италии тысячи людей выступили против мер бюджетной экономии правительства\r\n\r\nИтальянский политик, лидер партии "Народ свободы" и крупный бизнесмен, чей холдинг Fininvest владеет разнородными активами - от медиа и строительного бизнеса до футбольного клуба Milan (всего около 150 компаний) трижды избирался на пост главы правительства (абсолютный рекорд для Италии): возглавлял правительство Италии в 1994 г., в 2001-2006 гг. и в 2008-2011 г. В ноябре 2011 г., потеряв большинство в парламенте страны, добровольно ушел в отставку. По данным журнала Forbes, в марте 2011 г. состояние Берлускони оценивалось в 7,8 млрд долл.\r\n\r\nСкандально известная фигура в стране - Берлускони многократно обвинялся в финансовых аферах, коррупции, даче взяток госчиновникам, участии в сексуальных скандалах, привлекался к суду. Часть инициированных против него уголовных процессов закончились тюремными сроками, которые, впрочем, предприимчивый политик ни разу не отсидел. Каждый раз Верховный суд аннулировал приговоры судов низших инстанций.\r\n',
          [Span(2, 13, 'PER'),
           Span(152, 163, '

## BIO

BIO разметка: B - begin, I - inner, O - outer. Преобразуем задачу разметки спанов в задачу классификации каждого слова.

In [0]:
from razdel import tokenize
from collections import namedtuple

Sample = namedtuple("Sample", "text,tokens,spans,labels")

samples = []
for text, spans in data:
    labels = []
    tokens = list(tokenize(text))
    for token in tokens:
        label = 0
        for span in spans:
            if token.start == span[0]:
                label = 1
            elif token.start > span[0] and token.stop <= span[1]:
                label = 2
        labels.append(label)
    sample = Sample(text, tokens, spans, labels)
    samples.append(sample)

show_box_markup(samples[0].text, samples[0].spans, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))
print(samples[0].labels)
print(len(samples))

BoxMarkup('\r\nМарио Монти подаст в отставку с должности премьер-министра Италии\r\nФото: Reuters Фото: Reuters Фото: Reuters\r\nФото: Reuters\r\n\r\nПремьер-министр Италии Марио Монти подтвердил намерение подать прошение об отставке после того, как парламент одобрит закон о государственном бюджете на 2013 г. Этот и другие вопросы глава кабинета обсудил в ходе двухчасовой беседы с Президентом страны Джорджо Наполитано, передает Reuters.\r\n\r\nОтметим, что ранее входившая в парламентскую коалицию партия "Народ свободы" отказала Монти в поддержке. В свою очередь 76-летний Сильвио Берлускони, который является лидером указанной партии, за несколько часов до заявления Монти сообщил прессе, что сохраняет премьерские амбиции и готов вновь бороться за пост главы правительства республики в 2013 году.\r\n\r\nВ случае если бюджет будет принят "быстро", Монти готов немедленно сложить с себя полномочия. Примечательно, что Президент имеет полномочия объявить о роспуске парламента и проведении досрочных выборов. Несмотря на то, что такой вариант является маловероятным, итальянские политологи рассматривают его в качестве возможного исхода ситуации.\r\n\r\nПолитологи заключили, что теперь Берлускони обрушится с жесткой критикой на действующий кабинет министров, проводящий политику жесткой экономии.\r\n\r\n    Читайте также: Берлускони намерен в 2013 г. снова баллотироваться на пост премьер-министра Италии\r\n\r\nБерлускони ушел в отставку немногим более года назад, после чего в стране с согласия парламентского большинства было составлено правительство технократов, которому было поручено проводить непопулярную экономическую политику.\r\n\r\nКак сообщалось, 26 октября с.г. суд приговорил Берлускони к году тюрьмы с учетом закона об амнистии. Отмечается, что документ был принят в 2006 г., и в соответствии с ним срок тюремного заключения бывшего итальянского премьера уменьшен с четырех лет до одного года. Речь идет о скандальном деле Mediaset, в рамках которого Берлускони обвинялся в масштабных финансовых махинациях. Берлускони также отрицает обвинения в сексе с несовершеннолетней проституткой, известной под прозвищем "Руби".\r\n\r\n    Читайте также: Адвокаты Берлускони обжалуют приговор экс-премьеру\r\n\r\nОтметим также, действующий секретарь левоцентристской Демократической партии Италии Пьер Луиджи Берсани станет кандидатом на пост премьер-министра страны. Именно его кандидатуру поддержало большинство сторонников партии на праймериз 2 декабря. На выборах Берсани обошел мэра Флоренции Маттео Ренци, считающегося одним из наиболее заметных политиков "новой волны" в Италии. Демократическая партия, созданная на основе коалиции "Оливковое дерево" бывшего премьер-министра страны Романо Проди и включающая в себя партии "Левые демократы" и "Маргерита", занимает 217 мест в 630-местном парламенте Италии и является одной из главных оппозиционных сил страны.\r\n\r\n    По теме: В Италии тысячи людей выступили против мер бюджетной экономии правительства\r\n\r\nИтальянский политик, лидер партии "Народ свободы" и крупный бизнесмен, чей холдинг Fininvest владеет разнородными активами - от медиа и строительного бизнеса до футбольного клуба Milan (всего около 150 компаний) трижды избирался на пост главы правительства (абсолютный рекорд для Италии): возглавлял правительство Италии в 1994 г., в 2001-2006 гг. и в 2008-2011 г. В ноябре 2011 г., потеряв большинство в парламенте страны, добровольно ушел в отставку. По данным журнала Forbes, в марте 2011 г. состояние Берлускони оценивалось в 7,8 млрд долл.\r\n\r\nСкандально известная фигура в стране - Берлускони многократно обвинялся в финансовых аферах, коррупции, даче взяток госчиновникам, участии в сексуальных скандалах, привлекался к суду. Часть инициированных против него уголовных процессов закончились тюремными сроками, которые, впрочем, предприимчивый политик ни разу не отсидел. Каждый раз Верховный суд аннулировал приговоры судов низших инстанций.\r\n',
          [Span(2, 13, 'PER'),
           Span(152, 163, '

[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

Бьём на выборки

In [0]:
import random
random.shuffle(samples)

train = samples[:700]
val = samples[700:850]
test = samples[850:]

In [0]:
char_set = ["<pad>", "<unk>"] + list({ch for sample in samples for token in sample.tokens for ch in token.text})
print(char_set)

['<pad>', '<unk>', 'Т', '”', 'ё', '7', '1', 'д', '>', '©', 'c', 'k', 'м', 'Х', ',', 'Л', 'l', '…', 'А', 'F', 'U', '|', '[', 'н', 's', 'J', 'u', 'ь', 'Q', '+', 'й', 'O', 'g', 'В', 'x', 'z', 'y', '“', 'e', 'щ', ']', 'r', 'П', 't', 'm', 'У', '%', ';', 'f', 'Ф', 'ы', 'Р', 'E', '!', 'B', 'Щ', 'd', 'ъ', 'Y', '*', 'И', 'ю', '3', 'I', 'P', 'R', '»', 'з', 'э', '€', 'ж', 'p', 'v', 'N', '<', 'п', 'M', 'o', 'Ц', 'Б', '\xad', 'ц', ':', 'Ё', '6', 'і', 'и', '"', 'Н', 'Я', 'ф', 'G', 'H', 'C', 'Z', '–', 'к', 'О', 'Э', 'i', 'a', 'Д', 'З', 'т', 'Ж', 'b', '.', 'h', 'Г', '/', '2', 'Ч', 'а', 'у', 'в', 'е', 'D', 'T', 'г', 'K', 'L', 'о', 'q', 'С', '(', 'р', '-', 'ш', 'w', '№', 'Ы', '=', '0', '?', 'л', ')', '_', 'Ъ', 'М', 'Й', '5', 'Ю', 'Ш', '#', 'б', 'S', '4', 'A', '«', 'я', 'Е', "'", '8', 'j', 'V', '$', '•', 'х', 'n', 'ч', '—', 'Ь', 'W', 'с', 'К', 'X', '&', '9']


Для каждого слова сохраняем его символьный состав, а в остальном старый добрый пайплайн

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import numpy as np

def get_next_gen_batch(samples, max_seq_len=500, max_char_seq_len=40, batch_size=32):
    indices = np.arange(len(samples))
    np.random.shuffle(indices)
    batch_begin = 0
    while batch_begin < len(samples):
        batch_indices = indices[batch_begin: batch_begin + batch_size]
        batch = []
        batch_labels = []
        batch_max_len = 0
        for data_ind in batch_indices:
            sample = samples[data_ind]
            inputs = []
            for token in sample.tokens[:max_seq_len]:
                chars = [char_set.index(ch) if ch in char_set else char_set.index("<unk>") for ch in token.text][:max_char_seq_len]
                chars += [0] * (max_char_seq_len - len(chars))
                inputs.append(chars)
            batch_max_len = max(batch_max_len, len(inputs))
            inputs += [[0]*max_char_seq_len] * (max_seq_len - len(inputs))
            batch.append(inputs)
            labels = sample.labels[:max_seq_len]
            labels += [0] * (max_seq_len - len(labels))
            batch_labels.append(labels)
        batch_begin += batch_size
        batch = torch.cuda.LongTensor(batch)[:, :batch_max_len]
        labels = torch.cuda.LongTensor(batch_labels)[:, :batch_max_len]
        yield batch_indices, batch, labels


def train_gen_model(model, train_samples, val_samples, epochs_count=10, 
                    loss_every_nsteps=1000, lr=0.01, save_path="model.pt", device_name="cuda",
                    early_stopping=True,
                    batch_size=32,
                    crf=False):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss().cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (_, batch, batch_labels) in enumerate(get_next_gen_batch(train, batch_size=batch_size)):
            
            if crf:
              loss = model(batch, batch_labels)
            else :    
              logits = model(batch) # Прямой проход
              logits = logits.transpose(1, 2)
              loss = loss_function(logits, batch_labels) # Подсчёт ошибки
            
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()

        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (_, batch, batch_labels) in enumerate(get_next_gen_batch(val)):
            if crf:
              loss = model(batch, batch_labels)
              val_total_loss += loss
            else :    
              logits = model(batch) # Прямой проход
              logits = logits.transpose(1, 2)
              val_total_loss += loss_function(logits, batch_labels) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if early_stopping and prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

## Бесконтекстная модель

In [0]:
import torch
from torch import nn

class SuperSimpleModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=16, classes_count=3, char_max_seq_len=40):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.out_layer = nn.Linear(char_max_seq_len * char_embedding_dim, classes_count)

    def forward(self, inputs):
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        output = self.out_layer.forward(projections)
        return output


model = SuperSimpleModel(len(char_set))
train_gen_model(model, train, val, epochs_count=30, early_stopping=False, lr=0.02)

Trainable params: 4611
Epoch = 0, Avg Train Loss = 0.0184, Avg val loss = 0.3265, Time = 4.66s
Epoch = 1, Avg Train Loss = 0.0047, Avg val loss = 0.1220, Time = 4.57s
Epoch = 2, Avg Train Loss = 0.0023, Avg val loss = 0.0930, Time = 4.60s
Epoch = 3, Avg Train Loss = 0.0020, Avg val loss = 0.0846, Time = 4.58s
Epoch = 4, Avg Train Loss = 0.0018, Avg val loss = 0.0821, Time = 4.62s
Epoch = 5, Avg Train Loss = 0.0018, Avg val loss = 0.0774, Time = 4.59s
Epoch = 6, Avg Train Loss = 0.0017, Avg val loss = 0.0771, Time = 4.59s
Epoch = 7, Avg Train Loss = 0.0017, Avg val loss = 0.0736, Time = 4.49s
Epoch = 8, Avg Train Loss = 0.0016, Avg val loss = 0.0740, Time = 4.64s
Epoch = 9, Avg Train Loss = 0.0016, Avg val loss = 0.0729, Time = 4.53s
Epoch = 10, Avg Train Loss = 0.0016, Avg val loss = 0.0737, Time = 4.58s
Epoch = 11, Avg Train Loss = 0.0016, Avg val loss = 0.0730, Time = 4.60s


KeyboardInterrupt: ignored

## Метрики

Можно использовать как классические мультиклассификационнные метрики, так и метрики специально для NER.

Например, число точных и частичных совпадений спанов, пропущшенных и лишних спанов.

In [0]:
def get_spans(labels, tokens):
    spans = []
    for i, label in enumerate(labels):
        if label == 1:
            spans.append((tokens[i].start, tokens[i].stop, "PER"))
        elif label == 2:
            spans[-1] = (spans[-1][0], tokens[i].stop, spans[-1][-1])
    return spans


def calc_metrics(true_labels, predicted_labels, samples):
    one_tp = 0
    one_fp = 0
    one_fn = 0
    for true, predicted in zip(true_labels, predicted_labels):
        for l1, l2 in zip(true, predicted):
            if l1 == 1 and l2 == 1:
                one_tp += 1
            elif l1 != 1 and l2 == 1:
                one_fp += 1
            elif l1 == 1 and l2 !=1:
                one_fn += 1
    if one_tp + one_fp == 0:
        print("No positives!")
    else:
        print("1 Precision: {}, 1 Recall: {}".format(float(one_tp)/(one_tp + one_fp), float(one_tp)/(one_tp + one_fn)))

    exact = 0
    partial = 0
    missing = 0
    spurius = 0
    for (true, predicted), sample in zip(zip(true_labels, predicted_labels), samples):
        true_spans = get_spans(true, sample.tokens)
        predicted_spans = get_spans(predicted, sample.tokens)
        for true_span in true_spans:
            is_missing = True
            for predicted_span in predicted_spans:
                if true_span == predicted_span:
                    exact += 1
                    is_missing = False
                    break
                ts = true_span[0]
                te = true_span[1]
                ps = predicted_span[0]
                pe = predicted_span[1]
                # ts te ps pe
                # ps pe ts te
                if ts <= te <= ps <= pe or ps <= pe <= ts <= te:
                    continue
                is_missing = False
                partial += 1
                break
            if is_missing:
                missing += 1
        for predicted_span in predicted_spans:
            is_missing = True
            for true_span in true_spans:
                if true_span == predicted_span:
                    is_missing = False
                    break
                ts = true_span[0]
                te = true_span[1]
                ps = predicted_span[0]
                pe = predicted_span[1]
                if ts <= te <= ps <= pe or ps <= pe <= ts <= te:
                    continue
                is_missing = False
                break
            if is_missing:
                spurius += 1
    print("Exact: {}, partial: {}, missing: {}, spurius: {}".format(exact, partial, missing, spurius))
            


def predict(model, samples, crf=False):
    model.eval()
    true_labels = []
    predicted_labels = []
    all_indices = []
    for _, (indices, batch, batch_labels) in enumerate(get_next_gen_batch(samples)):
        if crf:
          em, mask = model.eval_forward(batch, batch_labels)
          plabels = model.CRF.decode(em, mask=mask)
        else:
          logits = model(batch)
          plabels = logits.max(dim=2)[1]
        # Убираем неконсистентность
        for sample_num, sample in enumerate(plabels):
            for word_num, label in enumerate(sample):
                if label != 2:
                    continue
                if word_num == 0:
                    plabels[sample_num][word_num] = 0
                    continue
                if sample[word_num - 1] == 0:
                    plabels[sample_num][word_num] = 0
        true_labels.extend(batch_labels)
        predicted_labels.extend(plabels)
        all_indices.extend(indices)
    samples = [samples[index] for index in all_indices]
    calc_metrics(true_labels, predicted_labels, samples)
    show_box_markup(samples[0].text, get_spans(predicted_labels[0], samples[0].tokens), palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))

In [0]:
predict(model, test)

1 Precision: 0.6365740740740741, 1 Recall: 0.527831094049904
Exact: 420, partial: 476, missing: 667, spurius: 267


BoxMarkup('Акционеры "ВымпелКома" на внеочередном собрании досрочно изберут новый совет директоров.\r\n\r\nАкционеры ОАО "ВымпелКом" (торговая марка "Билайн") сегодня на внеочередном общем собрании, как планируется, рассмотрят вопрос о досрочном прекращении полномочий членов совета директоров и избрании его нового состава. Также планируется утвердить "Эрнст энд Янг" в качестве аудитора финансовой отчетности компании по US GAAP, обсудить вступление компании в Союз операторов связи LTE, рассмотреть вопрос о вознаграждении членов совета директоров.\r\n\r\nНапомним, сейчас в совет директоров "ВымпелКома" входят президент и генеральный директор VimpelCom Ltd. Александр Изосимов, главный финансовый директор VimpelCom Ltd. Хендрик ван Дален, вице-президент департамента управления активами Altimo Алексей Гаврилов, главный управляющий директор Telenor Serbia Къелл Мортен Йонсен, главный юрисконсульт компании Джеффри Дэвид Мак-Ги.\r\n\r\nКак сообщалось ранее, с 30 июня 2011г. А.Изосимов должен покинуть компанию.\r\n\r\nОАО "ВымпелКом" - российская компания, предоставляющая услуги сотовой и фиксированной связи, проводного и беспроводного высокоскоростного доступа в интернет физическим и юридическим лицам под торговой маркой "Билайн". Компания ведет свою деятельность на всей территории России, а также в Киргизии, Казахстане, на Украине, в Таджикистане, Узбекистане, Армении, Грузии, Камбодже и Вьетнаме. Штаб-квартира организации расположена в Москве.\r\n\r\n',
          [Span(644, 653, 'PER'),
           Span(654, 662, 'PER'),
           Span(719, 724, 'PER'),
           Span(781, 797, 'PER'),
           Span(843, 848, 'PER'),
           Span(856, 862, 'PER'),
           Span(894, 901, 'PER'),
           Span(902, 907, 'PER'),
           Span(958, 959, 'PER'),
           Span(960, 968, 'PER'),
           Span(1286, 1294, 'PER')])

## Контекстная модель: LSTM над конкатенацией

In [0]:
import torch
from torch import nn

class LstmModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3, lstm_embedding_dim=8, char_max_seq_len=40):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.2)
        self.lstm_layer = nn.LSTM(char_embedding_dim * char_max_seq_len, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)

    def forward(self, inputs):
        batch_size = inputs.size(0)
        seq_len = inputs.size(1)
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        output = self.out_layer.forward(output)
        return output

model = LstmModel(len(char_set))
train_gen_model(model, train, val, epochs_count=50, early_stopping=False, lr=0.05, batch_size=70)

Trainable params: 6011
Epoch = 0, Avg Train Loss = 0.0078, Avg val loss = 0.5260, Time = 5.75s
Epoch = 1, Avg Train Loss = 0.0042, Avg val loss = 0.3311, Time = 5.70s
Epoch = 2, Avg Train Loss = 0.0030, Avg val loss = 0.2592, Time = 5.82s
Epoch = 3, Avg Train Loss = 0.0026, Avg val loss = 0.2312, Time = 5.75s
Epoch = 4, Avg Train Loss = 0.0024, Avg val loss = 0.2170, Time = 5.75s
Epoch = 5, Avg Train Loss = 0.0023, Avg val loss = 0.2081, Time = 5.67s
Epoch = 6, Avg Train Loss = 0.0022, Avg val loss = 0.2029, Time = 5.89s
Epoch = 7, Avg Train Loss = 0.0022, Avg val loss = 0.1962, Time = 5.73s
Epoch = 8, Avg Train Loss = 0.0021, Avg val loss = 0.1952, Time = 5.71s
Epoch = 9, Avg Train Loss = 0.0021, Avg val loss = 0.1952, Time = 5.79s
Epoch = 10, Avg Train Loss = 0.0021, Avg val loss = 0.2023, Time = 5.78s
Epoch = 11, Avg Train Loss = 0.0021, Avg val loss = 0.1940, Time = 5.68s
Epoch = 12, Avg Train Loss = 0.0020, Avg val loss = 0.1901, Time = 5.82s
Epoch = 13, Avg Train Loss = 0.0020, A

KeyboardInterrupt: ignored

In [0]:
predict(model, test)

No positives!
Exact: 0, partial: 0, missing: 1563, spurius: 0


BoxMarkup('Новым директором по качеству АВТОВАЗа назначили Л.Фофана\r\n\r\nБывший директор программы легких грузовых автомобилей Лоран Фофана сменил Паскаля Фелтена на посту директора по качеству ОАО "АВТОВАЗ", сообщили в пресс-службе компании. Приказ о назначении Л.Фофана подписал президент АВТОВАЗа Игорь Комаров.\r\n\r\nВозглавлявший ранее дирекцию по качеству П.Фелтен завершает свою работу на АВТОВАЗе в связи с истечением срока контракта. Он возглавит предприятие по производству крупногабаритных автомобилей в компании Renault.\r\n\r\nЛ.Фофана с 1994г. работал в Renault. С 1994г. по 2000г. был заместителем директора по производству силовых агрегатов; в 2000-2005гг. занимал пост директора проекта двигателей (занимался улучшением качества существующих двигателей в сотрудничестве со службами проектирования и производства, а также работал с поставщиками). В 2005-2011гг. стал директором программы легких грузовых автомобилей.\r\n\r\nОАО "АВТОВАЗ" (Тольятти) зарегистрировано 5 января 1993г. Автоконцерн выпускает автомобили семейств Lada Priora, Kalina, Samara, а также "классику".\r\n\r\nВ настоящее время доли между основными акционерами АВТОВАЗа распределяются следующим образом: ГК "Ростехнологии" - 25%, ИК "Тройка Диалог" - 25% и Renault - 25%.\r\n\r\n',
          [])

## Контекстная модель: LSTM над CharFF

## Задание 0
Сделайте полносвзяный слой с активацией над конкатенацией символьных эмбедов

In [0]:
from torch import nn

class CharFFLstmModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3, word_embedding_dim=16, lstm_embedding_dim=16, char_max_seq_len=40):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.4)
        self.char_activation = nn.Linear(char_embedding_dim * char_max_seq_len, word_embedding_dim)
        self.lstm_layer = nn.LSTM(word_embedding_dim, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        batch_size = inputs.size(0)
        seq_len = inputs.size(1)
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        projections = self.relu(self.char_activation(projections))
        projections = self.dropout(projections)
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        output = self.out_layer.forward(output)
        return output

model = CharFFLstmModel(len(char_set))
train_gen_model(model, train, val, epochs_count=50, early_stopping=False, lr=0.02)

Trainable params: 4963
Epoch = 0, Avg Train Loss = 0.0072, Avg val loss = 0.2086, Time = 4.89s
Epoch = 1, Avg Train Loss = 0.0049, Avg val loss = 0.1947, Time = 4.94s
Epoch = 2, Avg Train Loss = 0.0048, Avg val loss = 0.1906, Time = 5.18s
Epoch = 3, Avg Train Loss = 0.0048, Avg val loss = 0.1893, Time = 5.01s
Epoch = 4, Avg Train Loss = 0.0047, Avg val loss = 0.1878, Time = 4.97s
Epoch = 5, Avg Train Loss = 0.0046, Avg val loss = 0.1881, Time = 5.05s
Epoch = 6, Avg Train Loss = 0.0046, Avg val loss = 0.1739, Time = 5.19s
Epoch = 7, Avg Train Loss = 0.0044, Avg val loss = 0.1515, Time = 5.21s
Epoch = 8, Avg Train Loss = 0.0038, Avg val loss = 0.1266, Time = 5.08s
Epoch = 9, Avg Train Loss = 0.0033, Avg val loss = 0.1200, Time = 5.12s
Epoch = 10, Avg Train Loss = 0.0029, Avg val loss = 0.1040, Time = 4.87s
Epoch = 11, Avg Train Loss = 0.0026, Avg val loss = 0.0993, Time = 4.83s
Epoch = 12, Avg Train Loss = 0.0024, Avg val loss = 0.1168, Time = 4.87s
Epoch = 13, Avg Train Loss = 0.0023, A

KeyboardInterrupt: ignored

In [0]:
predict(model, test)

1 Precision: 0.8921568627450981, 1 Recall: 0.06066666666666667
Exact: 62, partial: 40, missing: 1398, spurius: 2


BoxMarkup('Медведев пообещал новые отставки в связи с терактом в "Домодедово"\r\n\r\nПрезидент РФ Дмитрий Медведев заявил на совещании по безопасности на транспорте, что отставки чиновников в связи с терактом в "Домодедово" будут продолжены. Об этом 26 января сообщает РИА Новости.\r\n\r\nГлава МВД РФ Рашид Нургалиев на совещании объявил об увольнении начальника линейного отдела внутренних дел "Домодедово" и двух его заместителей. Сам Медведев сказал, что подписал указ об отставке начальника управления на транспорте МВД по Центральному федеральному округу Андрея Алексеева. После этого Медведев заявил: "На этом все закончиться не должно, это как бы верхние начальники".\r\n\r\nВ ходе совещания Медведев напомнил, что в марте 2010 года подписал указ по обеспечению безопасности на транспорте. "Я хотел бы, чтобы мне сейчас доложили, как идет работа по указу, что уже сделано и в чем проблема", - сказал президент.\r\n\r\nКак сообщалось ранее, в начале совещания Медведев поручил Нургалиеву "тряхнуть" транспортную милицию с целью обеспечения безопасности.\r\n\r\nВзрыв в зоне прилета "Домодедово" произошел в 16:30 24 января. Бомбу подорвал террорист-смертник. Погибли 35 человек, более ста человек получили ранения. Возбуждены два уголовных дела - непосредственно о теракте и о необеспечении безопасности в аэропорту. 26 января в Москве и Московской области объявлен траур в память о жертвах теракта. ',
          [Span(542, 558, 'PER')])

## Задание 1.1
Сделайте то же самое, но с bidirectional LSTM на уровне символов

In [0]:
from torch import nn

class CharLstmModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3, word_embedding_dim=16, lstm_embedding_dim=16, char_max_seq_len=40):
        super().__init__()
        
        self.char_embedding_dim = char_embedding_dim
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.4)
        # self.char_activation = nn.Linear(char_embedding_dim * char_max_seq_len, word_embedding_dim)
        self.lstm_layer = nn.LSTM(char_embedding_dim, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(char_max_seq_len*lstm_embedding_dim, classes_count)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        batch_size = inputs.size(0)
        seq_len = inputs.size(1)
        projections = self.embeddings_layer.forward(inputs)
        wc = projections.size(1)
        projections = projections.reshape(projections.size(0), -1, self.char_embedding_dim)
        # print(projections.shape)
        # projections = self.relu(self.char_activation(projections))
        # projections = self.dropout(projections)
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        # print(output.shape)
        output = output.reshape(output.size(0), wc, -1)
        # print(output.shape)
        output = self.out_layer.forward(output)
        return output

model = CharLstmModel(len(char_set))
train_gen_model(model, train, val, epochs_count=25, early_stopping=False, lr=0.02)

Trainable params: 3491
Epoch = 0, Avg Train Loss = 0.0056, Avg val loss = 0.1830, Time = 27.46s
Epoch = 1, Avg Train Loss = 0.0041, Avg val loss = 0.1535, Time = 27.61s
Epoch = 2, Avg Train Loss = 0.0033, Avg val loss = 0.1213, Time = 27.30s
Epoch = 3, Avg Train Loss = 0.0027, Avg val loss = 0.0963, Time = 27.31s
Epoch = 4, Avg Train Loss = 0.0022, Avg val loss = 0.0799, Time = 27.89s
Epoch = 5, Avg Train Loss = 0.0019, Avg val loss = 0.0800, Time = 27.77s
Epoch = 6, Avg Train Loss = 0.0018, Avg val loss = 0.0723, Time = 27.73s
Epoch = 7, Avg Train Loss = 0.0016, Avg val loss = 0.0683, Time = 27.94s
Epoch = 8, Avg Train Loss = 0.0016, Avg val loss = 0.0674, Time = 27.58s
Epoch = 9, Avg Train Loss = 0.0015, Avg val loss = 0.0610, Time = 28.43s
Epoch = 10, Avg Train Loss = 0.0015, Avg val loss = 0.0615, Time = 27.92s
Epoch = 11, Avg Train Loss = 0.0014, Avg val loss = 0.0600, Time = 27.86s
Epoch = 12, Avg Train Loss = 0.0014, Avg val loss = 0.0606, Time = 28.17s
Epoch = 13, Avg Train Los

In [0]:
predict(model, test)

1 Precision: 0.8161512027491409, 1 Recall: 0.6333333333333333
Exact: 683, partial: 319, missing: 498, spurius: 101


BoxMarkup('С.Собянин уволил начальника московского здравоохранения\r\n\r\nМэр Москвы Сергей Собянин сегодня провел ряд перестановок в столичном руководстве. Интересно, что кадровые решения коснулись не только чиновников, назначенных при прежнем градоначальнике Юрии Лужкове, но и членов новой команды.\r\n\r\nС.Собянин сообщил сегодня на заседании правительства столицы об отставке главы Департамента здравоохранения Сергея Полякова в связи с переходом на новую должность. На какую работу ушел С.Поляков, пока не сообщается. Новым руководителем Департамента здравоохранения Москвы назначен Леонид Михайлович Печатников. Представляя нового главу департамента чиновникам, мэр отметил, что Л.Печатников - "профессор, с 2004г. и по настоящее время работает руководителем Европейского медицинского центра и возглавляет подразделение, связанное с подготовкой к Олимпийским играм в Сочи".\r\n\r\n"Надеюсь, с его приходом департамент заработает по-новому, с новой энергией", - добавил мэр. Он поручил Л.Печатникову сделать в своей работе упор на развитие детской медицины, "у которой больше всего проблем" и разобраться с тендерами на закупку лекарств и медицинского оборудования.\r\n\r\nНапомним, С.Собянин назначил ранее занимавшего должность первого заместителя руководителя Сергея Полякова главой Департамента здравоохранения Москвы 26 октября, он сменил на этом посту Андрея Сельцовского.\r\n\r\nОтметим, что 3 декабря С.Собянин вывел Департамент здравоохранения из сфер курируемых заместителем мэра Людмилой Швецовой и назначил Ольгу Голодец своим новым заместителем, контролирующим сферы образования и здравоохранения.\r\n\r\nСреди других сегодняшних кадровых решений мэра - увольнение зампреда Контрольного комитета Москвы Вячеслава Урюпина. Он освобожден от должности "по собственной инициативе".\r\n\r\nНа должность заместителя руководителя Департамента транспорта и связи Москвы назначен Юрий Мазикин. Заместителями руководителя Департамента имущества Москвы утверждены Сергей Шогуров, Вероника Бобровская и Валерий Ануприенко. Назначены четыре заместителя руководителя Департамента земельных ресурсов Москвы: Татьяна Найденова, Полина Афоничева, Ирина Ткачева и Людмила Останкова. Кроме того, на должность заместителя префекта Западного административного округа столицы назначен Владимир Говердовский.\r\n\r\nПост замглавы аппарата мэра и правительства Москвы и начальника управления референтуры мэрии занял Павел Чинилин.\r\n\r\n',
          [Span(0, 9, 'PER'),
           Span(59, 62, 'PER'),
           Span(70, 84, 'PER'),
           Span(246, 258, 'PER'),
           Span(290, 299, 'PER'),
           Span(398, 413, 'PER'),
           Span(475, 484, 'PER'),
           Span(571, 577, 'PER'),
           Span(578, 599, 'PER'),
           Span(668, 680, 'PER'),
           Span(970, 983, 'PER'),
           Span(1163, 1172, 'PER'),
           Span(1243, 1258, 'PER'),
           Span(1338, 1357, 'PER'),
           Span(1385, 1394, 'PER'),
           Span(1466, 1474, 'PER'),
           Span(1495, 1500, 'PER'),
           Span(1688, 1705, 'PER'),
           Span(1852, 1864, 'PER'),
           Span(1934, 1948, 'PER'),
           Span(1950, 1958, 'PER'),
           Span(1972, 1990, 'PER'),
           Span(2074, 2091, 'PER'),
           Span(2093, 2109, 'PER'),
           Span(2111, 2124, 'PER'),
           Span(2127, 2144, 'PER'),
           Span(2244, 2265, 'PER'),
           Span(2369, 2382, 'PER')])

## Задание 1.2
Сделайте то же самое, но со свёртками на уровне символов

In [0]:
from torch import nn

class CharCNNBiLstmModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3, word_embedding_dim=16, lstm_embedding_dim=16, char_max_seq_len=40):
        super().__init__()
        
        self.char_embedding_dim = char_embedding_dim
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.4)
        self.char_activation = nn.Conv2d(1, word_embedding_dim, kernel_size=(1, char_embedding_dim), padding=1)
        self.lstm_layer = nn.LSTM(48, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        batch_size = inputs.size(0)
        seq_len = inputs.size(1)
        projections = self.embeddings_layer.forward(inputs)
        bs = projections.size(0)
        wc = projections.size(1)
        # projections = projections.reshape(projections.size(0), 1, -1, self.char_embedding_dim)
        projections = projections.unsqueeze(2)
        # print(projections.shape)
        projections = projections.reshape(bs * wc, 1, 40, -1)
        # print(projections.shape)
        # projections = self.relu(self.char_activation(projections))
        # projections = self.dropout(projections)
        projections = self.char_activation(projections)
        # print(projections.shape)
        projections, _ = torch.max(projections, dim=2)
        # print(projections.shape)
        output = projections.reshape(bs, wc, -1)
        # print(output.shape)
        output, _= self.lstm_layer(output)
        output = self.dropout(output)
        # print(output.shape)
        output = self.out_layer.forward(output)
        return output

model = CharCNNBiLstmModel(len(char_set))
train_gen_model(model, train, val, epochs_count=25, early_stopping=False, lr=0.02)

Trainable params: 4515
Epoch = 0, Avg Train Loss = 0.0078, Avg val loss = 0.2043, Time = 6.12s
Epoch = 1, Avg Train Loss = 0.0047, Avg val loss = 0.1685, Time = 6.12s
Epoch = 2, Avg Train Loss = 0.0044, Avg val loss = 0.1609, Time = 6.10s
Epoch = 3, Avg Train Loss = 0.0040, Avg val loss = 0.1464, Time = 6.12s
Epoch = 4, Avg Train Loss = 0.0033, Avg val loss = 0.1095, Time = 6.09s
Epoch = 5, Avg Train Loss = 0.0026, Avg val loss = 0.0948, Time = 6.06s
Epoch = 6, Avg Train Loss = 0.0024, Avg val loss = 0.0909, Time = 6.14s
Epoch = 7, Avg Train Loss = 0.0023, Avg val loss = 0.0866, Time = 6.11s
Epoch = 8, Avg Train Loss = 0.0022, Avg val loss = 0.0865, Time = 6.04s
Epoch = 9, Avg Train Loss = 0.0022, Avg val loss = 0.0920, Time = 6.11s
Epoch = 10, Avg Train Loss = 0.0021, Avg val loss = 0.0828, Time = 6.18s
Epoch = 11, Avg Train Loss = 0.0019, Avg val loss = 0.0733, Time = 6.08s
Epoch = 12, Avg Train Loss = 0.0018, Avg val loss = 0.0634, Time = 6.11s
Epoch = 13, Avg Train Loss = 0.0016, A

In [0]:
predict(model, test)

1 Precision: 0.7871591908531222, 1 Recall: 0.5966666666666667
Exact: 851, partial: 137, missing: 512, spurius: 151


BoxMarkup('Избирательной кампанией "Единой России" займется Сергей Нарышкин\r\n\r\nРуководить избирательной кампанией "Единой России" будет глава администрации президента Сергей Нарышкин. Как пишет "Коммерсантъ", такое решение было принято в связи с тем, что список "ЕР" на выборах в Госдуму возглавил Дмитрий Медведев.\r\n\r\nПо данным издания, Нарышкин уже провел два совещания по поводу кампании. В частности, ему доложили о партийцах, которые будут участвовать в предвыборных дебатах.\r\n\r\nПресс-секретарь лидера "ЕР" Владимира Путина Дмитрий Песков подтвердил "Коммерсанту", что "основной акцент теперь" на президенте. Тем не менее, штаб Общероссийского народного фронта продолжает работать под руководством главы аппарата правительства Вячеслава Володина.\r\n\r\nКроме того, существует штаб "Единой России", который возглавляет секретарь генсовета партии Сергей Неверов. Сам Неверов отказался сказать, какую роль теперь выполняет его структура. "Ему нечего сказать", - заявили "Коммерсанту" в пресс-службе функционера.\r\n\r\nКак отмечает издание, о том, что вести кампанию "Единой России" будет именно администрация президента, свидетельствует включение в генсовет партии главы управления внутренней политики Константина Костина. В СМИ его называли человеком первого заместителя главы администрации Владислава Суркова.\r\n\r\nМежду тем "РБК daily" стали известны подробности финансирования кампании "Единой России". По сведениям издания, регионы практически не будут получать помощи из центра. Региональные кампании, как пишет издание, будут спонсировать бизнесмены, попавшие на проходные места. В среднем, по информации "РБК daily", предприниматели заплатили по пять миллионов евро.\r\n\r\nВыборы в Госдуму состоятся 4 декабря. То, что список "Единой России" на выборах возглавит президент Дмитрий Медведев стало известно на съезде партии 24 сентября. По словам главы государства, "ЕР" является наиболее близкой ему политической силой, несмотря на то, что он ее неоднократно критиковал. ',
          [Span(49, 78, 'PER'),
           Span(156, 171, 'PER'),
           Span(287, 303, 'PER'),
           Span(501, 532, 'PER'),
           Span(721, 739, 'PER'),
           Span(836, 850, 'PER'),
           Span(1187, 1206, 'PER'),
           Span(1277, 1295, 'PER'),
           Span(1761, 1777, 'PER')])

## Задание 1.3
Сделайте то же самое, но с CRF над головой

In [0]:
!pip install pytorch-crf

In [88]:
from torch import nn
from torchcrf import CRF

class CharCNNBiLstmModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3, word_embedding_dim=16, lstm_embedding_dim=16, char_max_seq_len=40):
        super().__init__()
        
        self.char_embedding_dim = char_embedding_dim
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.4)
        self.char_activation = nn.Conv2d(1, word_embedding_dim, kernel_size=(1, char_embedding_dim), padding=1)
        self.lstm_layer = nn.LSTM(48, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)
        self.relu = nn.ReLU()
        self.CRF = CRF(classes_count, batch_first=True)

    def forward(self, inputs, tags):
        
        batch_size = inputs.size(0)
        seq_len = inputs.size(1)
        projections = self.embeddings_layer.forward(inputs)
        bs = projections.size(0)
        wc = projections.size(1)
        # projections = projections.reshape(projections.size(0), 1, -1, self.char_embedding_dim)
        projections = projections.unsqueeze(2)
        # print(projections.shape)
        projections = projections.reshape(bs * wc, 1, 40, -1)
        # print(projections.shape)
        # projections = self.relu(self.char_activation(projections))
        # projections = self.dropout(projections)
        projections = self.char_activation(projections)
        # print(projections.shape)
        projections, _ = torch.max(projections, dim=2)
        # print(projections.shape)
        output = projections.reshape(bs, wc, -1)
        # print(output.shape)
        output, _= self.lstm_layer(output)
        output = self.dropout(output)
        # print(output.shape)
        output = self.out_layer.forward(output)

        mask = torch.zeros((inputs.shape[0], inputs.shape[1]), dtype = torch.uint8)
        for i in range(mask.shape[0]):
          for j in range(mask.shape[1]):
            if inputs[i][j].sum() >= 1:
              mask[i, j] = 1

        device = torch.device("cuda")
        mask = mask.to(device)

        output = -self.CRF(output, tags, reduction='mean', mask=mask)
        return output

    def eval_forward(self, inputs, tags):
        batch_size = inputs.size(0)
        seq_len = inputs.size(1)
        projections = self.embeddings_layer.forward(inputs)
        bs = projections.size(0)
        wc = projections.size(1)
        # projections = projections.reshape(projections.size(0), 1, -1, self.char_embedding_dim)
        projections = projections.unsqueeze(2)
        # print(projections.shape)
        projections = projections.reshape(bs * wc, 1, 40, -1)
        # print(projections.shape)
        # projections = self.relu(self.char_activation(projections))
        # projections = self.dropout(projections)
        projections = self.char_activation(projections)
        # print(projections.shape)
        projections, _ = torch.max(projections, dim=2)
        # print(projections.shape)
        output = projections.reshape(bs, wc, -1)
        # print(output.shape)
        output, _= self.lstm_layer(output)
        output = self.dropout(output)
        # print(output.shape)
        output = self.out_layer.forward(output)


        
        mask = torch.zeros((inputs.shape[0], inputs.shape[1]), dtype = torch.uint8)
        for i in range(mask.shape[0]):
          for j in range(mask.shape[1]):
            if inputs[i][j].sum() >= 1:
              mask[i, j] = 1

        device = torch.device("cuda")
        mask = mask.to(device)
        


        return output, mask

model = CharCNNBiLstmModel(len(char_set))
train_gen_model(model, train, val, epochs_count=50, early_stopping=False, lr=0.02, crf=True)

Trainable params: 4530
Epoch = 0, Avg Train Loss = 2.3713, Avg val loss = 75.3236, Time = 38.51s
Epoch = 1, Avg Train Loss = 1.7133, Avg val loss = 67.8249, Time = 37.46s
Epoch = 2, Avg Train Loss = 1.5143, Avg val loss = 63.2991, Time = 38.28s
Epoch = 3, Avg Train Loss = 1.3912, Avg val loss = 59.1591, Time = 37.77s
Epoch = 4, Avg Train Loss = 1.3199, Avg val loss = 57.2932, Time = 36.64s
Epoch = 5, Avg Train Loss = 1.2824, Avg val loss = 55.6300, Time = 38.02s
Epoch = 6, Avg Train Loss = 1.2564, Avg val loss = 55.7411, Time = 38.03s
Epoch = 7, Avg Train Loss = 1.2337, Avg val loss = 54.3748, Time = 37.79s
Epoch = 8, Avg Train Loss = 1.2142, Avg val loss = 53.4952, Time = 38.12s
Epoch = 9, Avg Train Loss = 1.1730, Avg val loss = 50.4621, Time = 38.24s
Epoch = 10, Avg Train Loss = 1.0949, Avg val loss = 48.3805, Time = 38.07s
Epoch = 11, Avg Train Loss = 1.0498, Avg val loss = 45.6042, Time = 38.31s
Epoch = 12, Avg Train Loss = 1.0162, Avg val loss = 43.9468, Time = 37.71s
Epoch = 13, 

In [89]:
train_gen_model(model, train, val, epochs_count=50, early_stopping=False, lr=0.02, crf=True)

Trainable params: 4530
Epoch = 0, Avg Train Loss = 0.2492, Avg val loss = 11.2964, Time = 37.84s
Epoch = 1, Avg Train Loss = 0.2350, Avg val loss = 10.6377, Time = 37.66s
Epoch = 2, Avg Train Loss = 0.2294, Avg val loss = 10.8173, Time = 38.03s
Epoch = 3, Avg Train Loss = 0.2266, Avg val loss = 10.0112, Time = 37.86s
Epoch = 4, Avg Train Loss = 0.2228, Avg val loss = 9.8630, Time = 37.81s
Epoch = 5, Avg Train Loss = 0.2176, Avg val loss = 9.8484, Time = 36.95s
Epoch = 6, Avg Train Loss = 0.2174, Avg val loss = 9.7550, Time = 37.27s
Epoch = 7, Avg Train Loss = 0.2144, Avg val loss = 10.1669, Time = 37.33s
Epoch = 8, Avg Train Loss = 0.2119, Avg val loss = 9.6427, Time = 37.89s
Epoch = 9, Avg Train Loss = 0.2107, Avg val loss = 10.0398, Time = 37.72s
Epoch = 10, Avg Train Loss = 0.2145, Avg val loss = 9.4291, Time = 37.89s
Epoch = 11, Avg Train Loss = 0.2106, Avg val loss = 9.3550, Time = 37.78s


KeyboardInterrupt: ignored

In [92]:
predict(model, test, crf=True)

1 Precision: 0.8773055332798717, 1 Recall: 0.7293333333333333
Exact: 1069, partial: 98, missing: 333, spurius: 85


BoxMarkup('Медведев пообещал новые отставки в связи с терактом в "Домодедово"\r\n\r\nПрезидент РФ Дмитрий Медведев заявил на совещании по безопасности на транспорте, что отставки чиновников в связи с терактом в "Домодедово" будут продолжены. Об этом 26 января сообщает РИА Новости.\r\n\r\nГлава МВД РФ Рашид Нургалиев на совещании объявил об увольнении начальника линейного отдела внутренних дел "Домодедово" и двух его заместителей. Сам Медведев сказал, что подписал указ об отставке начальника управления на транспорте МВД по Центральному федеральному округу Андрея Алексеева. После этого Медведев заявил: "На этом все закончиться не должно, это как бы верхние начальники".\r\n\r\nВ ходе совещания Медведев напомнил, что в марте 2010 года подписал указ по обеспечению безопасности на транспорте. "Я хотел бы, чтобы мне сейчас доложили, как идет работа по указу, что уже сделано и в чем проблема", - сказал президент.\r\n\r\nКак сообщалось ранее, в начале совещания Медведев поручил Нургалиеву "тряхнуть" транспортную милицию с целью обеспечения безопасности.\r\n\r\nВзрыв в зоне прилета "Домодедово" произошел в 16:30 24 января. Бомбу подорвал террорист-смертник. Погибли 35 человек, более ста человек получили ранения. Возбуждены два уголовных дела - непосредственно о теракте и о необеспечении безопасности в аэропорту. 26 января в Москве и Московской области объявлен траур в память о жертвах теракта. ',
          [Span(0, 8, 'PER'),
           Span(83, 99, 'PER'),
           Span(283, 298, 'PER'),
           Span(415, 427, 'PER'),
           Span(542, 558, 'PER'),
           Span(572, 580, 'PER'),
           Span(677, 685, 'PER'),
           Span(940, 948, 'PER')])

# NER из коробки

https://github.com/natasha/natasha

https://github.com/deepmipt/DeepPavlov

https://pypi.org/project/polyglot/

http://www.pullenti.ru/

In [101]:
!pip install natasha

     |████████████████████████████████| 778kB 45.9MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 


## Задание 2
Оцените готовые модели из natasha и deeppavlov-ner на нашем тестовом датасете

In [109]:
from natasha import NamesExtractor
	

text = '''
Простите, еще несколько цитат из приговора. «…Отрицал существование
Иисуса и пророка Мухаммеда», «наделял Иисуса Христа качествами
ожившего мертвеца — зомби» [и] «качествами покемонов —
представителей бестиария японской мифологии, тем самым совершил
преступление, предусмотренное статьей 148 УК РФ
'''
extractor = NamesExtractor()
matches = extractor(text)
for match in matches:
    print(match.span, match.fact)

# (69, 75) Name(first='иисус', last=None, middle=None, nick=None)
# (86, 95) Name(first='мухаммед', last=None, middle=None, nick=None)
# (107, 120) Name(first='иисус', last='христос', middle=None, nick=None)

[69, 75) Name(first='иисус', middle=None, last=None, nick=None)
[86, 95) Name(first='мухаммед', middle=None, last=None, nick=None)
[107, 120) Name(first='иисус', middle=None, last='христос', nick=None)


In [112]:
true_labels = []
predicted_labels = []
all_indices = []

def compare_spans(spans_pred, spans_true):
  one_tp = 0
  one_fp = 0
  one_fn = 0

  exact = 0
  partial = 0
  missing = 0
  spurius = 0
  for (predicted_spans, true_spans) in zip(spans_pred, spans_true):
      for true_span in true_spans:
          is_missing = True
          for predicted_span in predicted_spans:
              if true_span == predicted_span:
                  exact += 1
                  is_missing = False
                  break
              ts = true_span[0]
              te = true_span[1]
              ps = predicted_span[0]
              pe = predicted_span[1]
              # ts te ps pe
              # ps pe ts te
              if ts <= te <= ps <= pe or ps <= pe <= ts <= te:
                  continue
              is_missing = False
              partial += 1
              break
          if is_missing:
              missing += 1
      for predicted_span in predicted_spans:
          is_missing = True
          for true_span in true_spans:
              if true_span == predicted_span:
                  is_missing = False
                  break
              ts = true_span[0]
              te = true_span[1]
              ps = predicted_span[0]
              pe = predicted_span[1]
              if ts <= te <= ps <= pe or ps <= pe <= ts <= te:
                  continue
              is_missing = False
              break
          if is_missing:
              spurius += 1
  print("Exact: {}, partial: {}, missing: {}, spurius: {}".format(exact, partial, missing, spurius))
    

pred_spans = []
true_spans = []
for exmpl in test:
    pred_spans.append([(s.span[0], s.span[1], "PER") for s in extractor(exmpl.text)])
    true_spans.append(exmpl.spans)

compare_spans(pred_spans, true_spans)



# samples = [samples[index] for index in all_indices]
# calc_metrics(true_labels, predicted_labels, samples)
# show_box_markup(samples[0].text, get_spans(predicted_labels[0], samples[0].tokens), palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))


Exact: 1019, partial: 218, missing: 294, spurius: 57


In [93]:
!wget http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v2.tar.gz
!tar -xzvf rubert_cased_L-12_H-768_A-12_v2.tar.gz

--2019-12-17 10:41:11--  http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v2.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662018287 (631M) [application/octet-stream]
Saving to: ‘rubert_cased_L-12_H-768_A-12_v2.tar.gz’

rubert_cased_L-12_H 100%[===================>] 631.35M  6.57MB/s    in 90s     

2019-12-17 10:42:43 (7.05 MB/s) - ‘rubert_cased_L-12_H-768_A-12_v2.tar.gz’ saved [662018287/662018287]

rubert_cased_L-12_H-768_A-12_v2/
rubert_cased_L-12_H-768_A-12_v2/bert_config.json
rubert_cased_L-12_H-768_A-12_v2/vocab.txt
rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt.data-00000-of-00001
rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt.index


In [94]:
!pip install deeppavlov

     |████████████████████████████████| 737kB 48.7MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 2.8MB 42.1MB/s 
     |████████████████████████████████| 25.2MB 389kB/s 
     |████████████████████████████████| 8.0MB 67.4MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 10.1MB 55.1MB/s 
     |████████████████████████████████| 6.7MB 61.8MB/s 
     |████████████████████████████████| 317kB 57.6MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 2.1MB 52.0MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 163kB 61.6MB/s 
     |████████████████████████████████| 17.3MB 52.3MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 112kB 72.3MB/s 
 

In [95]:
!python -m deeppavlov install ner_rus_bert

email-validator not installed, email fields will be treated as str.
To install, run: pip install email-validator
2019-12-17 10:43:34.521 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'ner_rus_bert' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_rus_bert.json'
     |████████████████████████████████| 109.2MB 134kB/s 
     |████████████████████████████████| 3.2MB 33.4MB/s 
     |████████████████████████████████| 491kB 64.6MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-

In [96]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2019-12-17 10:44:10.322 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz to /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz
100%|██████████| 666M/666M [01:40<00:00, 6.64MB/s]
2019-12-17 10:45:50.597 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz archive into /root/.deeppavlov/downloads/bert_models
2019-12-17 10:45:57.486 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz to /root/.deeppavlov/ner_rus_bert_v1.tar.gz
100%|██████████| 1.32G/1.32G [03:40<00:00, 5.97MB/s]
2019-12-17 10:49:38.137 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/ner_rus_bert_v1.tar.gz archive into /root/.deeppavlov/models
[nltk_data] Downloading package punkt to /ro

2019-12-17 10:49:55.67 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_rus_bert/tag.dict]


Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Ins

2019-12-17 10:50:21.617 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_rus_bert/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_rus_bert/model


In [0]:
def calc_metrics_bert(true_spans, predicted_spans):
    exact = 0
    partial = 0
    missing = 0
    spurius = 0
    for true_span in true_spans:
        is_missing = True
        for predicted_span in predicted_spans:
            if true_span == predicted_span:
                exact += 1
                is_missing = False
                break
            ts = true_span[0]
            te = true_span[1]
            ps = predicted_span[0]
            pe = predicted_span[1]
            # ts te ps pe
            # ps pe ts te
            if ts <= te <= ps <= pe or ps <= pe <= ts <= te:
                continue
            is_missing = False
            partial += 1
            break
        if is_missing:
            missing += 1
    for predicted_span in predicted_spans:
        is_missing = True
        for true_span in true_spans:
            if true_span == predicted_span:
                is_missing = False
                break
            ts = true_span[0]
            te = true_span[1]
            ps = predicted_span[0]
            pe = predicted_span[1]
            if ts <= te <= ps <= pe or ps <= pe <= ts <= te:
                continue
            is_missing = False
            break
        if is_missing:
            spurius += 1
    # print("Exact: {}, partial: {}, missing: {}, spurius: {}".format(exact, partial, missing, spurius))
    return exact, partial, missing, spurius

In [111]:
from razdel import sentenize
from tqdm import tqdm_notebook as tqdm
aexact = 0
apartial = 0
amissing = 0
aspurius = 0

for sample in tqdm(test):
    true_spans = sample.spans
    markup = ner_model([sample.text[:1200]])
    tokenized_text = markup[0][0]
    tags = markup[1][0]
    start = 0
    predicted_spans = []
    for token, tag in zip(tokenized_text, tags):
        begin = sample.text.find(token, start)
        end = begin + len(token)
        start = end
        if tag.endswith("PER"):
            if tag[0] == "B":
                predicted_spans.append((begin, end, "PER"))
            else:
                if len(predicted_spans) == 0: continue
                predicted_spans[-1] = (predicted_spans[-1][0], end, "PER")

    # predicted_spans = [(match.span[0], match.span[1], "PER") for match in extractor(sample.text)]
    # show_box_markup(sample.text, predicted_spans, palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))
    exact, partial, missing, spurius = calc_metrics_bert(true_spans, predicted_spans)
    aexact += exact
    apartial += partial
    amissing += missing
    aspurius += spurius

print(aexact, apartial, amissing, aspurius)


1093 34 404 13
